In [41]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import Resize, Compose
import segmentation_models_pytorch as smp

from src.models.BaselineModel import BaselineModel
from src.evaluation.evaluate_result import evaluate_result
from src.datasets.AerialDroneSemanticSegmentationDataset import (
    AerialDroneSemanticSegmentationDataset,
)

from src.datasets.utils.ResizeToDivisibleBy32 import ResizeToDivisibleBy32

## Prepare environment

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
VAL_SIZE = 0.2
BATCH_SIZE = 1
SEED = 42
AERIAL_DRONE_DATASET_PATH = (
    "data/AerialDroneSemanticSegmentationDataset/dataset/semantic_drone_dataset"
)
IMAGE_SIZE = 576

In [5]:
train_dataset = AerialDroneSemanticSegmentationDataset(
    AERIAL_DRONE_DATASET_PATH,
    transforms=[Compose([Resize(IMAGE_SIZE), ResizeToDivisibleBy32()])],
)
print(len(train_dataset))

400


## Sanity check data

In [6]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [7]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

torch.Size([1, 3, 576, 864])
torch.Size([1, 1, 576, 864])


In [8]:
to_pil_transform = transforms.ToPILImage()

In [9]:
img = to_pil_transform(images.squeeze())

In [10]:
# img.show()

In [11]:
msk = to_pil_transform(masks.squeeze())

In [12]:
# msk.show()

## Run example model on single image

In [13]:
model = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=23,  # model output channels (number of classes in your dataset)
    activation="softmax",
).to(device)

In [14]:
baseline_model = BaselineModel(classes=23).to(device)

In [15]:
torch.cuda.empty_cache()

In [16]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    # output = (output > 0.5).float()

    output_baseline = baseline_model(images.to(device))
    # output_baseline = (output_baseline > 0.5).float()

torch.Size([1, 3, 576, 864])
torch.Size([1, 1, 576, 864])


d:\__repos\aerial_segmentation\venv\lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [17]:
print(output.shape)

torch.Size([1, 23, 576, 864])


In [18]:
print(output_baseline.shape)

torch.Size([1, 23, 576, 864])


In [19]:
output = torch.argmax(output, dim=1)
output_baseline = torch.argmax(output_baseline, dim=1)

In [20]:
print(output.shape)
print(output_baseline.shape)

torch.Size([1, 576, 864])
torch.Size([1, 576, 864])


In [21]:
evaluate_result(output.squeeze(), masks.squeeze(), mode="multiclass", num_classes=23)

{'iou': tensor(0.0418),
 'f1': tensor(0.0803),
 'accuracy': tensor(0.9200),
 'recall': tensor(0.0803)}

In [22]:
evaluate_result(
    output_baseline.squeeze(), masks.squeeze(), mode="multiclass", num_classes=23
)

{'iou': tensor(0.0222),
 'f1': tensor(0.0435),
 'accuracy': tensor(0.9168),
 'recall': tensor(0.0435)}

## Run example model on dataset

In [ ]:
model.eval()
metrics_dict = {
    "iou": [],
    "f1": [],
    "accuracy": [],
    "recall": [],
}

In [ ]:
with torch.no_grad():
    for images, masks in train_loader:
        output = model(images.to(device))
        output = torch.argmax(output, dim=1)

        iter_metrics = evaluate_result(
            output.squeeze(), masks.squeeze(), mode="multiclass", num_classes=23
        )
        for key in metrics_dict.keys():
            metrics_dict[key].append(iter_metrics[key])

In [ ]:
print(metrics_dict)

In [ ]:
print("Mean metrics")
for key in metrics_dict.keys():
    print(key, np.mean(metrics_dict[key]))

# UNET++

## Run example model on single image

In [23]:
model = smp.UnetPlusPlus(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=23,  # model output channels (number of classes in your dataset)
    activation="softmax",
).to(device)

In [24]:
baseline_model = BaselineModel()

In [25]:
torch.cuda.empty_cache()

In [26]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    output = torch.argmax(output, dim=1)

    output_baseline = baseline_model(images.to(device))
    output_baseline = torch.argmax(output_baseline, dim=1)

torch.Size([1, 3, 576, 864])
torch.Size([1, 1, 576, 864])


d:\__repos\aerial_segmentation\venv\lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [27]:
print(output.shape)

torch.Size([1, 576, 864])


In [28]:
print(output_baseline.shape)

torch.Size([1, 576, 864])


In [29]:
to_pil_transform = transforms.ToPILImage()

In [30]:
img = to_pil_transform(images.squeeze())

In [31]:
# img.show()

In [32]:
msk = to_pil_transform(masks.squeeze())

In [34]:
# msk.show()

In [38]:
unique, counts = np.unique(output.to('cpu'), return_counts=True)
print(dict(zip(unique, counts)))

{0: 2454, 2: 49, 3: 48, 4: 34874, 6: 1958, 7: 2, 8: 18, 9: 18, 10: 1, 12: 136, 13: 223, 15: 6878, 17: 17830, 18: 386058, 19: 5, 20: 4532, 22: 42580}


In [39]:
outp = to_pil_transform(output.int().squeeze().cpu())

In [42]:
# plt.imsave("assets/aerial-drone-example-unet-plus-plus-output.jpeg", np.array(outp))

In [40]:
# outp.show()

In [44]:
evaluate_result(output, masks, mode="multiclass", num_classes=23)

{'iou': tensor(0.0032),
 'f1': tensor(0.0064),
 'accuracy': tensor(0.9136),
 'recall': tensor(0.0064)}

## DeepLabV3

## Run example model on single image

In [45]:
model = smp.DeepLabV3(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=23,
    activation="softmax"
).to(device)

In [46]:
baseline_model = BaselineModel()

In [47]:
torch.cuda.empty_cache()

In [48]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    output = torch.argmax(output, dim=1)

    output_baseline = baseline_model(images.to(device))
    output_baseline = torch.argmax(output_baseline, dim=1)

torch.Size([1, 3, 576, 864])
torch.Size([1, 1, 576, 864])


d:\__repos\aerial_segmentation\venv\lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [49]:
print(output.shape)

torch.Size([1, 576, 864])


In [50]:
print(output_baseline.shape)

torch.Size([1, 576, 864])


In [51]:
to_pil_transform = transforms.ToPILImage()

In [52]:
img = to_pil_transform(images.squeeze())

In [ ]:
# img.show()

In [53]:
msk = to_pil_transform(masks.squeeze())

In [ ]:
# msk.show()

In [54]:
outp = to_pil_transform(output.int().squeeze().cpu())

In [56]:
plt.imsave("assets/aerial-drone-example-deep-lab-v3-output.jpeg", np.array(outp))

In [55]:
# outp.show()

In [57]:
evaluate_result(output, masks, mode="multiclass", num_classes=23)

{'iou': tensor(0.0002),
 'f1': tensor(0.0003),
 'accuracy': tensor(0.9131),
 'recall': tensor(0.0003)}

## DeepLabV3+

## Run example model on single image

In [58]:
model = smp.DeepLabV3Plus(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=23,
    activation="softmax"
).to(device)

In [59]:
baseline_model = BaselineModel()

In [60]:
torch.cuda.empty_cache()

In [61]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    output = torch.argmax(output, dim=1)

    output_baseline = baseline_model(images.to(device))
    output_baseline = torch.argmax(output_baseline, dim=1)

torch.Size([1, 3, 576, 864])
torch.Size([1, 1, 576, 864])


d:\__repos\aerial_segmentation\venv\lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [62]:
print(output.shape)

torch.Size([1, 576, 864])


In [63]:
print(output_baseline.shape)

torch.Size([1, 576, 864])


In [64]:
to_pil_transform = transforms.ToPILImage()

In [65]:
img = to_pil_transform(images.squeeze())

In [66]:
# img.show()

In [67]:
msk = to_pil_transform(masks.squeeze())

In [69]:
# msk.show()

In [70]:
outp = to_pil_transform(output.int().squeeze().cpu())

In [71]:
plt.imsave("assets/aerial-drone-example-deep-lav-v3-plus-output.jpeg", np.array(outp))

In [73]:
# outp.show()

In [74]:
evaluate_result(output, masks, mode="multiclass", num_classes=23)

{'iou': tensor(0.0006),
 'f1': tensor(0.0012),
 'accuracy': tensor(0.9131),
 'recall': tensor(0.0012)}